In [1]:
# Script Name: V101 (Manual Trading - Delta Neutral Two-Leg Execution)

import ccxt
import time
import datetime
from zoneinfo import ZoneInfo
import json
import os

# API Keys (Bybit and Bitget only for this trial)
BYBIT_API_KEY = 'I6m01uNAWiensHbIBA'
BYBIT_API_SECRET = 'vkaviS9xw903NJPncMtHgGfyiWB4mnhSJTyK'
BITGET_API_KEY = 'bg_b0a9d721607623216a87b066a3df34c3'
BITGET_API_SECRET = '6c4a5016562624b6d7f3b1c59563d4a1a573f630e0e49422fbf509399e56918b'
BITGET_API_PASSWORD = 'BgSecure789Trade2025'



In [2]:
# Settings
SPREAD_THRESHOLD = 0.4 / 100
MIN_VOLUME = 50000
CHECK_INTERVAL = 1
TRADE_USDT_SIZE = 30
ADD_POSITION_USDT = 15
LOCAL_TIMEZONE = ZoneInfo("Asia/Jakarta")

# Init
TRADING_FEES = {'Bybit': 0.001, 'Bitget': 0.0008}
TRADE_FILE = 'open_trades.json'
futures_markets_cache = {}

bybit = ccxt.bybit({'apiKey': BYBIT_API_KEY, 'secret': BYBIT_API_SECRET, 'enableRateLimit': True, 'options': {'defaultType': 'spot'}})
bitget = ccxt.bitget({'apiKey': BITGET_API_KEY, 'secret': BITGET_API_SECRET, 'password': BITGET_API_PASSWORD, 'enableRateLimit': True, 'options': {'defaultType': 'spot'}})
exchanges = {'Bybit': bybit, 'Bitget': bitget}

# Persistence

def load_open_trades():
    if os.path.exists(TRADE_FILE):
        with open(TRADE_FILE, 'r') as f:
            return json.load(f)
    return {}

def save_open_trades(trades):
    with open(TRADE_FILE, 'w') as f:
        json.dump(trades, f, indent=2)

open_trades = load_open_trades()

# Helper Functions

def find_futures_symbol(spot_symbol, futures_list):
    base = spot_symbol.split('/')[0]
    match = [f for f in futures_list if f.startswith(f"{base}/USDT") or f.startswith(f"{base}-USDT")]
    return match[0] if match else None

def get_price(exchange, symbol):
    try:
        return exchange.fetch_ticker(symbol).get("last", None)
    except:
        return None

def get_best_bid_ask(exchange, symbol):
    try:
        order_book = exchange.fetch_order_book(symbol)
        best_bid = order_book['bids'][0][0] if order_book['bids'] else None
        best_ask = order_book['asks'][0][0] if order_book['asks'] else None
        return best_bid, best_ask
    except:
        return None, None

def get_tradable_tokens(exchange, exchange_name):
    try:
        markets = exchange.load_markets()
        tradable_tokens = []
        futures_markets_cache[exchange_name] = [s for s in markets if ":USDT" in s or "-USDT" in s]
        for s, market in markets.items():
            if exchange_name == "Bitget" and not market.get('spot', False): continue
            if '/USDT' in s and ':USDT' not in s and '-USDT' not in s:
                volume = exchange.fetch_ticker(s).get('quoteVolume', 0)
                if volume and volume >= MIN_VOLUME:
                    tradable_tokens.append(s)
        return tradable_tokens
    except:
        return []

# Manual confirmation

def ask_permission_to_trade(symbol, spread, exchange_name, spot_price, futures_price):
    timestamp = datetime.datetime.now(LOCAL_TIMEZONE).strftime('%Y-%m-%d %H:%M:%S')
    print(f"\n🕒 Time: {timestamp}")
    print(f"📊 Spot Price: {spot_price:.4f}, Futures Price: {futures_price:.4f}")
    confirm = input(f"🚨 Opportunity detected on {exchange_name} for {symbol} | Spread: {spread:.4%}. Execute trade? (y/n): ")
    return confirm.lower() == 'y'

# Two-leg execution

def execute_two_leg_trade(exchange_name, symbol, usdt_size):
    try:
        exchange_spot = exchanges[exchange_name]
        futures_list = futures_markets_cache.get(exchange_name, [])
        futures_symbol = find_futures_symbol(symbol, futures_list)
        if not futures_symbol:
            print("❌ Futures symbol not found.")
            return

        spot_price = get_price(exchange_spot, symbol)
        amount = usdt_size / spot_price

        # Init futures client
        if exchange_name == 'Bybit':
            fut = ccxt.bybit({'apiKey': BYBIT_API_KEY, 'secret': BYBIT_API_SECRET, 'enableRateLimit': True, 'options': {'defaultType': 'future'}})
        else:
            fut = ccxt.bitget({'apiKey': BITGET_API_KEY, 'secret': BITGET_API_SECRET, 'password': BITGET_API_PASSWORD, 'enableRateLimit': True, 'options': {'defaultType': 'swap'}})
        fut.set_leverage(1, futures_symbol)

        # Get best prices
        spot_bid, _ = get_best_bid_ask(exchange_spot, symbol)
        _, future_ask = get_best_bid_ask(fut, futures_symbol)

        print(f"🛒 Spot Buy @ {spot_bid}, 📉 Futures Sell @ {future_ask}, Amount: {amount:.6f}")

        confirm = input("Confirm two-leg trade execution? (y/n): ")
        if confirm.lower() != 'y':
            print("❌ Trade cancelled by user.")
            return

        spot_order = exchange_spot.create_order(symbol=symbol, type='limit', side='buy', amount=amount, price=spot_bid)
        fut_order = fut.create_order(symbol=futures_symbol, type='limit', side='sell', amount=amount, price=future_ask)

        print(f"✅ Spot order placed: {spot_order['id']}, Futures order placed: {fut_order['id']}")

        # Save trade
        trade_id = f"{symbol.replace('/', '')}_{int(time.time())}"
        open_trades[trade_id] = {
            "symbol": symbol,
            "exchange": exchange_name,
            "amount": amount,
            "spot_price": spot_bid,
            "futures_price": future_ask,
            "entry_spread": (future_ask - spot_bid) / spot_bid,
            "timestamp": datetime.datetime.now().isoformat()
        }
        save_open_trades(open_trades)

    except Exception as e:
        print(f"❌ Execution Error: {e}")

# Arbitrage Opportunity Scanner

def check_arbitrage_opportunity(exchange, exchange_name, symbol):
    spot_price = get_price(exchange, symbol)
    futures_list = futures_markets_cache.get(exchange_name, [])
    futures_symbol = find_futures_symbol(symbol, futures_list)
    if not futures_symbol:
        return
    futures_price = get_price(exchange, futures_symbol)
    if not spot_price or not futures_price:
        return

    spread = (futures_price - spot_price) / spot_price
    net_profit = spread - (TRADING_FEES[exchange_name] * 2)

    if net_profit >= SPREAD_THRESHOLD:
        if ask_permission_to_trade(symbol, spread, exchange_name, spot_price, futures_price):
            execute_two_leg_trade(exchange_name, symbol, TRADE_USDT_SIZE)

# Main Loop

def main():
    while True:
        try:
            for exchange_name, exchange in exchanges.items():
                tokens = get_tradable_tokens(exchange, exchange_name)
                for token in tokens:
                    check_arbitrage_opportunity(exchange, exchange_name, token)
            time.sleep(CHECK_INTERVAL)
        except Exception as e:
            print(f"❌ Main Loop Error: {e}")
            time.sleep(5)

if __name__ == "__main__":
    main()


🕒 Time: 2025-04-14 22:36:00
📊 Spot Price: 0.0643, Futures Price: 0.0648


KeyboardInterrupt: Interrupted by user